# LangChain Chatbot (with chat history)

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

import pandas as pd
pd.set_option('display.max_colwidth', None)

In [2]:
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredHTMLLoader, UnstructuredPDFLoader
from langchain import PromptTemplate

In [3]:
import langchain
langchain.debug = True

### load data and setup vector store

In [4]:
documents = []
for file in os.listdir('data/test_documents'):
    if file.endswith('.pdf'):
        pdf_path = './data/test_documents/' + file
        print(f'Loading {pdf_path}')
        loader = UnstructuredPDFLoader(pdf_path)
        documents.extend(loader.load())
    elif file.endswith('.html'):
        doc_path = './data/test_documents/' + file
        print(f'Loading {doc_path}')
        loader = UnstructuredHTMLLoader(doc_path)
        documents.extend(loader.load())

Loading ./data/test_documents/Extension of Deadlines Master Business Infromatics.html
Loading ./data/test_documents/Extension of Deadlines MMDS.html
Loading ./data/test_documents/General Questions Master Business Informatics.html
Loading ./data/test_documents/General Questions MMDS.html
Loading ./data/test_documents/Learning Agreements Master Business Informatics.html
Loading ./data/test_documents/Learning Agreements MMDS.html
Loading ./data/test_documents/Master Business Informatics info start page.html
Loading ./data/test_documents/Master Thesis Information.html
Loading ./data/test_documents/MMDS info start page.html
Loading ./data/test_documents/Modue_Catalog_MSc_Wifo_23_24.pdf
Loading ./data/test_documents/Module_Catalog_Appendix_MMDS_23_24.pdf
Loading ./data/test_documents/Module_Catalog_MMDS_23_24.pdf
Loading ./data/test_documents/PO_MMDS_20.pdf
Loading ./data/test_documents/PO_MSc_Wifo_18.pdf
Loading ./data/test_documents/Recognition of Coursework and Examinations Master Busines

In [6]:
# split documents into text chunks

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunked_documents = text_splitter.split_documents(documents)

In [ ]:
# create chroma vector db with OpenAIEmbeddings

vectordb = Chroma.from_documents(
  chunked_documents,
  embedding=OpenAIEmbeddings(),
  persist_directory='./storage_langchain'
)
vectordb.persist()

### create QA Chain

In [8]:
template = """

If the question does not provide a specific study program, dont answer the question and ask what the student is studying.
Else use the following pieces of context to answer the question at the end.

To answer the question execute these steps:
1 - list the context
2 - focus on words like "optional" or "can" for your answer
3 - answer the question. Do not use information outside of the context to answer the question.

Your answer should have this format:

context:
answer:

------------------------
Context: {context}

Question: {question}

"""

custom_prompt = PromptTemplate.from_template(template)

In [16]:
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=ChatOpenAI(model='gpt-3.5-turbo', temperature=0),
    retriever=vectordb.as_retriever(search_kwargs={'k': 5}),
    return_source_documents=True,
    combine_docs_chain_kwargs={"prompt": custom_prompt}
)

# used to store previous questions and answers
chat_history = []

### Test

In [ ]:
query = "Do I have to do any fundamental courses?"

result = qa_chain({'question': query, 'chat_history': chat_history})
chat_history.append((query, result['answer']))
print(result['answer'])

In [ ]:
query = "Does that apply if i am studying the master of data science?"

result = qa_chain({'question': query, 'chat_history': chat_history})
chat_history.append((query, result['answer']))
print(result['answer'])

In [ ]:
query = "Does the same apply if I am studying Business Informatics?"

result = qa_chain({'question': query, 'chat_history': chat_history})
chat_history.append((query, result['answer']))
print(result['answer'])